1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다, 2. 문서를 쪼갠다

In [1]:
%pip install --upgrade --quiet  docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./tax.docx")
documents = loader.load_and_split(text_splitter=text_splitter)

In [11]:
len(documents)

220

# 3. 임베딩

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [9]:
%pip install langchain-chroma

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 9.3 MB/s eta 0:00:0031m9.4 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 11.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.2 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 12.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.7 MB/s eta 0:00:00
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53726 sha256=520d5f63c5245a601b17a5c67911984bc60cf93e0689e682de265a89d60e7bfe
  Stored in directory: /Users/choichiwon/Library/Caches/pip/wheels/e1/26/51/d0bffb

In [3]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory="./chroma", collection_name='chroma-tax')
database = Chroma(collection_name='chroma-tax', persist_directory='./chroma', embedding_function=embedding)

# 4. 질문이 있을 때 벡터 데이터베이스에 유사도 검색

In [4]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3)

In [16]:
retrieved_docs

[Document(metadata={'source': './tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n\n다. 삭제<2014. 12. 2

# 5. 유사도 검색으로 가져온 문서를 질문과 함께 LLM에게 전달

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [7]:
ai_response = llm.invoke(prompt)

In [8]:
ai_response.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 먼저 과세표준을 계산하고, 그 다음으로 기본세율을 적용해야 합니다. 여기서는 기본적인 계산 방식을 설명드리겠습니다.\n\n1. **총 급여에서 각종 공제 항목을 차감**:\n    - **근로소득공제**: 연봉 5천만원에 해당하는 근로소득공제 금액을 계산합니다.\n    - **인적공제 및 기타 공제**: 인적공제 (기본공제, 추가공제 등)와 기타 공제 항목 (보험료 공제, 의료비 공제, 교육비 공제 등)을 적용합니다.\n\n2. **과세표준 계산**:\n    - 총 급여액에서 근로소득공제와 인적공제 및 기타 공제를 차감하여 과세표준을 산출합니다.\n\n3. **기본세율 적용**:\n    - 과세표준에 기본세율을 적용하여 산출세액을 계산합니다.\n\n4. **세액공제 및 감면**:\n    - 산출세액에서 세액공제 및 감면 항목 (근로소득세액공제 등)을 차감하여 최종 산출세액을 계산합니다.\n\n**예시 계산**:\n\n1. **근로소득공제 계산**:\n    - 연봉 5천만원에 대한 근로소득공제는 다음과 같습니다:\n      ```\n      총급여액 5천만원에 대한 근로소득공제 = 1,200만원 + (총급여액 - 4,500만원) * 15%\n      = 1,200만원 + (5,000만원 - 4,500만원) * 15%\n      = 1,200만원 + 75만원\n      = 1,275만원\n      ```\n\n2. **과세표준 계산**:\n    - 연봉 5천만원에서 근로소득공제 1,275만원을 차감:\n      ```\n      총급여액 5천만원 - 근로소득공제 1,275만원 = 3,725만원\n      ```\n    - 인적공제 및 기타 공제 항목을 적용합니다. 여기서는 기본공제 (본인 150만원)만 적용한다고 가정:\n      ```\n      과세표준 = 3,725만원 - 기본공제 150만원 = 3,575만원\n      ```\n\n3. **기본세율 적용**:\n    -

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본세율을 적용해야 합니다. 기본세율은 과세표준에 따라 다르며, 다음과 같습니다:\n\n1. 과세표준 1,200만원 이하: 6%\n2. 과세표준 1,200만원 초과 4,600만원 이하: 15%\n3. 과세표준 4,600만원 초과 8,800만원 이하: 24%\n4. 과세표준 8,800만원 초과 1억5천만원 이하: 35%\n5. 과세표준 1억5천만원 초과 3억원 이하: 38%\n6. 과세표준 3억원 초과 5억원 이하: 40%\n7. 과세표준 5억원 초과: 42%\n\n연봉 5천만원의 경우, 과세표준은 다음과 같이 계산됩니다:\n\n1. 연봉 5천만원에서 근로소득공제와 각종 공제를 차감한 금액이 과세표준이 됩니다.\n2. 근로소득공제는 다음과 같습니다:\n   - 연봉 5천만원의 경우: 3,600만원 + (연봉 - 4,500만원) * 5% = 3,600만원 + (5,000만원 - 4,500만원) * 5% = 3,600만원 + 25만원 = 3,625만원\n\n따라서, 과세표준은 5천만원 - 3,625만원 = 1,375만원입니다.\n\n이제, 기본세율표를 적용하여 소득세를 계산합니다:\n\n- 과세표준 1,200만원 이하 부분: 1,200만원 * 6% = 72만원\n- 과세표준 1,200만원 초과 1,375만원 이하 부분: (1,375만원 - 1,200만원) * 15% = 175만원 * 15% = 26.25만원\n\n따라서, 총 소득세는 72만원 + 26.25만원 = 98.25만원입니다.\n\n연봉 5천만원인 직장인의 소득세는 대략 98.25만원이 됩니다. 다만, 이 계산은 근로소득공제와 기본세율만을 고려한 것이며, 실제 소득세는 각종 세액공제, 보험료 공제 등 다른 공제를 고려해야 합니다. 정확한 금액은 국세청 홈택스 등의 소득세 계산기를 이용해 보시는 것이 좋습니다.'

# 5-1. 직접 프롬프트를 만들지 말고, RetrievalQA, langchainhub 이용해서 간편하게 질문과 문서 던지기

In [9]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/choichiwon/.pyenv/versions/helloworld/lib/python3.10/site-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/choichiwon/.pyenv/versions/helloworld/lib/python3.10/site-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [11]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [14]:
ai_message = qa_chain({"query": query})

/var/folders/54/2vnvnn154r5_5fwkj7dvqtn00000gn/T/ipykernel_25660/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query": query})


In [15]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 기본세율이 적용됩니다. 기본세율은 소득 구간에 따라 다르며, 연봉 5천만원의 경우 대략 15%에서 24% 사이의 세율이 적용될 수 있습니다. 정확한 금액은 상세한 소득 공제와 세액 공제에 따라 달라질 수 있습니다.'}